# Лабораторная работа 2

## Приведение задачи линейного программирования к канонической и нормальной формам

Реализуем два алгоритма:
1. **Приведение к нормальной форме** — все ограничения вида $\leq$, целевой функционал максимизируется, все переменные $\geq 0$.
2. **Приведение к канонической форме** — все ограничения вида $=$, целевой функционал максимизируется, все переменные $\geq 0$.

In [1]:
import numpy as np

## Вспомогательные функции для отображения задачи ЛП

In [ ]:
def _int(v):
    """Приводит float к int, если значение целое."""
    return int(v) if v == int(v) else v


def format_expr(coeffs, names):
    """Форматирует линейное выражение: 3·x1 + 2·x2 - x3"""
    terms = []
    for c, name in zip(coeffs, names):
        if c == 0:
            continue
        v = _int(c)
        if not terms:
            if v == 1:    terms.append(name)
            elif v == -1: terms.append(f'-{name}')
            else:         terms.append(f'{v}·{name}')
        else:
            a = _int(abs(c))
            sign = '+ ' if v > 0 else '- '
            if a == 1: terms.append(f'{sign}{name}')
            else:      terms.append(f'{sign}{a}·{name}')
    return ' '.join(terms) if terms else '0'


def print_problem(c, d, A, b, r=None, sigma=None):
    """Выводит задачу ЛП в читаемом виде."""
    n = len(c)
    names = [f'x{j+1}' for j in range(n)]
    sign_map = {'<=': '≤', '>=': '≥', '=': '='}

    print(f'  {format_expr(c, names)} → {d}')
    for i in range(len(b)):
        s = sign_map.get(r[i], r[i]) if r else '='
        print(f'  {format_expr(A[i], names)} {s} {_int(b[i])}')

    if sigma:
        sm = {'>=0': '≥ 0', '<=0': '≤ 0', 'free': '∈ ℝ'}
        print(f'  {", ".join(f"{names[j]} {sm[sigma[j]]}" for j in range(n))}')


def print_matrices(c, A, b):
    """Выводит результирующие матрицы c, A, b."""
    ci, Ai, bi = c.astype(int), A.astype(int), b.astype(int)
    print(f'c = {ci}\n\nA =\n{Ai}\n\nb = {bi}')

## Исходная задача (3)

$$
\begin{cases}
-x_1 + x_2 - 2x_3 \to \min \\
x_1 + x_3 \leq 1 \\
x_1 - x_3 \geq 2 \\
x_1 + x_2 = 10 \\
x_1 \geq 0,\; x_2 \leq 0,\; x_3 \text{ — без ограничений}
\end{cases}
$$

Выделим составные части задачи: вектор $c$, направление $d$, матрицу $A$, вектор $b$, знаки ограничений $r$ и ограничения на знак переменных $\sigma$.

In [3]:
c = np.array([-1, 1, -2], dtype=float)
d = 'min'

A = np.array([
    [1, 0,  1],
    [1, 0, -1],
    [1, 1,  0]
], dtype=float)

b = np.array([1, 2, 10], dtype=float)

r = ['<=', '>=', '=']
sigma = ['>=0', '<=0', 'free']

print('Исходная задача:')
print_problem(c, d, A, b, r, sigma)

Исходная задача:
  -x1 + x2 - 2·x3 → min
  x1 + x3 ≤ 1
  x1 - x3 ≥ 2
  x1 + x2 = 10
  x1 ≥ 0, x2 ≤ 0, x3 ∈ ℝ


## Приведение к нормальной форме

**Алгоритм:**

1. Если $d = \min$, умножаем $c$ на $-1$, ставим $d = \max$.
2. Каждое ограничение «$=$» заменяем парой «$\leq$» и «$\geq$» (дублируем строку $A$ и элемент $b$).
3. Каждое ограничение «$\geq$» умножаем на $-1$ (строку $A$ и элемент $b$), получаем «$\leq$».
4. Если $x_j \leq 0$ (неположительная), заменяем $x_j = -x'_j$: умножаем $j$-й столбец $A$ и $c_j$ на $-1$.
5. Если $x_j$ без ограничений на знак, заменяем $x_j = x_j^+ - x_j^-$: добавляем столбец $-A_j$ и элемент $-c_j$.

In [4]:
def to_normal_form(c, d, A, b, r, sigma):
    c, A, b = c.copy(), A.copy(), b.copy()
    r, sigma = list(r), list(sigma)

    # Шаг 1: min → max
    if d == 'min':
        c = -c

    # Шаг 2: = → пара (≤, ≥)
    i = 0
    while i < len(r):
        if r[i] == '=':
            A = np.insert(A, i + 1, A[i], axis=0)
            b = np.insert(b, i + 1, b[i])
            r[i] = '<='
            r.insert(i + 1, '>=')
            i += 2
        else:
            i += 1

    # Шаг 3: ≥ → ≤ (умножение строки на -1)
    for i in range(len(r)):
        if r[i] == '>=':
            A[i] = -A[i]
            b[i] = -b[i]
            r[i] = '<='

    # Шаг 4: σ = ≤0 → замена x = -x'
    for j in range(len(sigma)):
        if sigma[j] == '<=0':
            A[:, j] = -A[:, j]
            c[j] = -c[j]
            sigma[j] = '>=0'

    # Шаг 5: σ = free → замена x = x⁺ - x⁻
    j = 0
    while j < len(sigma):
        if sigma[j] == 'free':
            neg_col = -A[:, j].reshape(-1, 1)
            A = np.hstack([A[:, :j+1], neg_col, A[:, j+1:]])
            c = np.concatenate([c[:j+1], [-c[j]], c[j+1:]])
            sigma[j] = '>=0'
            sigma.insert(j + 1, '>=0')
            j += 2
        else:
            j += 1

    return c, A, b

In [8]:
cn, An, bn = to_normal_form(c, d, A, b, r, sigma)

n_norm = len(cn)
print('Нормальная форма задачи:')
print_problem(cn, 'max', An, bn,
              r=['<='] * len(bn),
              sigma=['>=0'] * n_norm)
print()
print_matrices(cn, An, bn)

Нормальная форма задачи:
  x1 + x2 + 2·x3 - 2·x4 → max
  x1 + x3 - x4 ≤ 1
  -x1 + x3 - x4 ≤ -2
  x1 - x2 ≤ 10
  -x1 + x2 ≤ -10
  x1 ≥ 0, x2 ≥ 0, x3 ≥ 0, x4 ≥ 0

c = [ 1  1  2 -2]

A =
[[ 1  0  1 -1]
 [-1  0  1 -1]
 [ 1 -1  0  0]
 [-1  1  0  0]]

b = [  1  -2  10 -10]


## Приведение к канонической форме

**Алгоритм:**

1. Если $d = \min$, умножаем $c$ на $-1$, ставим $d = \max$.
2. Каждое ограничение «$\leq$» превращаем в «$=$», добавляя балансовую переменную $s \geq 0$: прибавляем столбец $e_i$ к $A$.
3. Каждое ограничение «$\geq$» превращаем в «$=$», вычитая балансовую переменную $s \geq 0$: прибавляем столбец $-e_i$ к $A$.
4. Если $x_j \leq 0$ (неположительная), заменяем $x_j = -x'_j$: умножаем $j$-й столбец $A$ и $c_j$ на $-1$.
5. Если $x_j$ без ограничений на знак, заменяем $x_j = x_j^+ - x_j^-$: добавляем столбец $-A_j$ и элемент $-c_j$.

In [9]:
def to_canonical_form(c, d, A, b, r, sigma):
    c, A, b = c.copy(), A.copy(), b.copy()
    r, sigma = list(r), list(sigma)
    m = len(r)

    # Шаг 1: min → max
    if d == 'min':
        c = -c

    # Шаг 2: ≤ → = (добавление балансовой переменной +s)
    for i in range(m):
        if r[i] == '<=':
            col = np.zeros((m, 1))
            col[i, 0] = 1.0
            A = np.hstack([A, col])
            c = np.append(c, 0.0)
            r[i] = '='
            sigma.append('>=0')

    # Шаг 3: ≥ → = (добавление балансовой переменной -s)
    for i in range(m):
        if r[i] == '>=':
            col = np.zeros((m, 1))
            col[i, 0] = -1.0
            A = np.hstack([A, col])
            c = np.append(c, 0.0)
            r[i] = '='
            sigma.append('>=0')

    # Шаг 4: σ = ≤0 → замена x = -x'
    for j in range(len(sigma)):
        if sigma[j] == '<=0':
            A[:, j] = -A[:, j]
            c[j] = -c[j]
            sigma[j] = '>=0'

    # Шаг 5: σ = free → замена x = x⁺ - x⁻
    j = 0
    while j < len(sigma):
        if sigma[j] == 'free':
            neg_col = -A[:, j].reshape(-1, 1)
            A = np.hstack([A[:, :j+1], neg_col, A[:, j+1:]])
            c = np.concatenate([c[:j+1], [-c[j]], c[j+1:]])
            sigma[j] = '>=0'
            sigma.insert(j + 1, '>=0')
            j += 2
        else:
            j += 1

    return c, A, b

In [12]:
cc, Ac, bc = to_canonical_form(c, d, A, b, r, sigma)

n_canon = len(cc)
print('Каноническая форма задачи:')
print_problem(cc, 'max', Ac, bc,
              r=['='] * len(bc),
              sigma=['>=0'] * n_canon)
print()
print_matrices(cc, Ac, bc)

Каноническая форма задачи:
  x1 + x2 + 2·x3 - 2·x4 → max
  x1 + x3 - x4 + x5 = 1
  x1 - x3 + x4 - x6 = 2
  x1 - x2 = 10
  x1 ≥ 0, x2 ≥ 0, x3 ≥ 0, x4 ≥ 0, x5 ≥ 0, x6 ≥ 0

c = [ 1  1  2 -2  0  0]

A =
[[ 1  0  1 -1  1  0]
 [ 1  0 -1  1  0 -1]
 [ 1 -1  0  0  0  0]]

b = [ 1  2 10]


## Проверка результатов

Сравним полученные матрицы с ожидаемыми результатами из методички.

**Нормальная форма** (ожидается):
$$c = \begin{pmatrix} 1 \\ 1 \\ 2 \\ -2 \end{pmatrix}, \quad
A = \begin{pmatrix} 1 & 0 & 1 & -1 \\ -1 & 0 & 1 & -1 \\ 1 & -1 & 0 & 0 \\ -1 & 1 & 0 & 0 \end{pmatrix}, \quad
b = \begin{pmatrix} 1 \\ -2 \\ 10 \\ -10 \end{pmatrix}$$

**Каноническая форма** (ожидается):
$$c = \begin{pmatrix} 1 \\ 1 \\ 2 \\ -2 \\ 0 \\ 0 \end{pmatrix}, \quad
A = \begin{pmatrix} 1 & 0 & 1 & -1 & 1 & 0 \\ 1 & 0 & -1 & 1 & 0 & -1 \\ 1 & -1 & 0 & 0 & 0 & 0 \end{pmatrix}, \quad
b = \begin{pmatrix} 1 \\ 2 \\ 10 \end{pmatrix}$$

In [11]:
c_norm_expected = np.array([1, 1, 2, -2])
A_norm_expected = np.array([
    [ 1,  0,  1, -1],
    [-1,  0,  1, -1],
    [ 1, -1,  0,  0],
    [-1,  1,  0,  0]
])
b_norm_expected = np.array([1, -2, 10, -10])

c_canon_expected = np.array([1, 1, 2, -2, 0, 0])
A_canon_expected = np.array([
    [1,  0,  1, -1,  1,  0],
    [1,  0, -1,  1,  0, -1],
    [1, -1,  0,  0,  0,  0]
])
b_canon_expected = np.array([1, 2, 10])

assert np.allclose(cn, c_norm_expected), f'c (норм.) не совпадает: {cn}'
assert np.allclose(An, A_norm_expected), f'A (норм.) не совпадает:\n{An}'
assert np.allclose(bn, b_norm_expected), f'b (норм.) не совпадает: {bn}'

assert np.allclose(cc, c_canon_expected), f'c (канон.) не совпадает: {cc}'
assert np.allclose(Ac, A_canon_expected), f'A (канон.) не совпадает:\n{Ac}'
assert np.allclose(bc, b_canon_expected), f'b (канон.) не совпадает: {bc}'

print('✅ Все результаты совпадают с ожидаемыми!')

✅ Все результаты совпадают с ожидаемыми!
